<a href="https://colab.research.google.com/github/daman1209arora/cnn-mnist/blob/main/Copy_of_mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# **Classification using a normal neural network**

**Architecture:**

*   4 fully connected layers consisting of 100, 35, 23, and 10 neurons
*   Activation function used is relU for all but the last layer. In that last layer, softmax is used.


In [ ]:
class NNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 35)
        self.fc3 = nn.Linear(35, 23)
        self.fc4 = nn.Linear(23, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x
    

nn_model = NNModel()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(nn_model.parameters())

# **Code for setting up the training**

*   Batch size = 50
*   No of epochs = 5
*   Train-test split = 90:10
*   Optimiser = Adam



In [ ]:
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=torchvision.transforms.ToTensor())
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])

train_data_loader = torch.utils.data.DataLoader(train_data, 32, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = nn_model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    print("After epoch {}, loss: {}".format(epoch + 1, total_loss))
    correct = 0
    total = 0

    test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
    with torch.no_grad():
        for i, data in enumerate(test_data_loader, 0):
            images, labels = data
            images = torch.flatten(images, start_dim=1)
            output = nn_model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    print("Accuracy on the test set:", 100 * correct / total)
    total_loss = 0

After epoch 1, loss: 2838.7731684446335
Accuracy on the test set: 84.16666666666667
After epoch 2, loss: 2680.9523997306824
Accuracy on the test set: 92.9
After epoch 3, loss: 2562.3945639133453
Accuracy on the test set: 94.76666666666667
After epoch 4, loss: 2544.841208100319
Accuracy on the test set: 93.91666666666667
After epoch 5, loss: 2535.277930378914
Accuracy on the test set: 95.68333333333334


# **Classification using a Convolutional Neural Network**
**Architecture:**\
1) Convolutional layer(input channels: 1, output channels: 10, kernel size: 5)\
2) Max-pooling layer(kernel size: 2)\
3) Convolutional layer(input channels: 10, output channels: 15, kernel size: 5)\
4) Flatten the output channels to form a tensor of dimensions (batch_size, 240)\
5) 3 fully connected layers of 100, 35, and 10 neurons



In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=15, kernel_size=5)
        self.fc1 = nn.Linear(15 * 4 * 4, 100)
        self.fc2 = nn.Linear(100, 35)
        self.fc3 = nn.Linear(35, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 15 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x), dim=1)
    

cnn_model = CNNModel()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters())

# **Code for setting up the training**

*   Batch size = 50
*   No of epochs = 5
*   Train-test split = 90:10
*   Optimiser = Adam


In [ ]:
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=torchvision.transforms.ToTensor())
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])

train_data_loader = torch.utils.data.DataLoader(train_data, 50, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        #images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = cnn_model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    print("After epoch {}, loss: {}".format(epoch + 1, total_loss))
    correct = 0
    total = 0

    test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
    with torch.no_grad():
        for i, data in enumerate(test_data_loader, 0):
            images, labels = data
            #images = torch.flatten(images, start_dim=1)
            output = cnn_model(images)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    print("Accuracy on the test set:", 100 * correct / total)
    total_loss = 0

After epoch 1, loss: 1832.1089605093002
Accuracy on the test set: 85.85
After epoch 2, loss: 1696.1449193954468
Accuracy on the test set: 97.11666666666666
After epoch 3, loss: 1611.422259569168
Accuracy on the test set: 97.65
After epoch 4, loss: 1603.175191283226
Accuracy on the test set: 98.08333333333333
After epoch 5, loss: 1600.010668873787
Accuracy on the test set: 98.08333333333333


# **Using resnet18 for a Transfer Learning model**

In [ ]:
import torchvision.models as models 

resnet18 = models.vgg16(pretrained=True)

for param in resnet18.parameters():
    param.requires_grad = False

# the 2nd last layer of resnet18 consists of 512 activations
# we will use those(not that last layer) to perform transfer learning 
class FCLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(512, 100)
        self.fc2 = nn.Linear(100, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x
    
#resnet18.fc = FCLayer()
#resnet18.fc = nn.Linear(512, 10)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters())


# **Training the model on MNIST**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
resnet18.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
data_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)), torchvision.transforms.ToTensor()])
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=data_transform)
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])
train_data_loader = torch.utils.data.DataLoader(train_data, 50, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], dim=1)
        #images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = resnet18(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 
        if(i % 100 == 99):
            print("After epoch {}, mini-batch {}, loss: {}".format(epoch + 1, i + 1, total_loss))
            correct = 0
            total = 0

            test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
            with torch.no_grad():
                for i, data in enumerate(test_data_loader, 0):
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    images = torch.cat([images, images, images], dim=1)
                    output = resnet18(images)
                    _, predicted = torch.max(output.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                
            print("Accuracy on the test set:", 100 * correct / total)
            total_loss = 0
    total_loss = 0

RuntimeError: ignored

# **Evaluating on test set**

In [ ]:
correct = 0
total = 0

test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
with torch.no_grad():
    for i, data in enumerate(test_data_loader, 0):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        images = torch.cat([images, images, images], dim=1)
        output = resnet18(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print("Accuracy on the test set:", 100 * correct / total)

Accuracy on the test set: 88.2
